In [1]:
#!git clone https://github.com/ultralytics/yolov5.git


In [2]:
%cd yolov5


D:\deep learning\rotsense\yolov5


In [3]:
#!pip install -r requirements.txt
#!pip install opencv-python


In [4]:
#!pip install torch torchvision


In [5]:
%cd "D:/deep learning/rotsense/yolov5"


D:\deep learning\rotsense\yolov5


In [6]:
import os
import sys

# Path to YOLOv5 clone directory
yolo_path = r"D:\deep learning\rotsense\yolov5"
os.chdir(yolo_path)

# Add YOLOv5 to system path
if yolo_path not in sys.path:
    sys.path.append(yolo_path)


In [7]:
import torch
import cv2
import numpy as np
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression
from utils.torch_utils import select_device


In [8]:
import os
import sys

yolo_path = r"D:\deep learning\rotsense\yolov5"
os.chdir(yolo_path)
if yolo_path not in sys.path:
    sys.path.append(yolo_path)


In [9]:
import pathlib
import pickle
pathlib.PosixPath = pathlib.WindowsPath


In [10]:
# Use CPU or GPU if available
device = select_device('')  # '' = auto

weights_path = r"D:\deep learning\rotsense\yolov5\best.pt"
 

# Initialize model
model = DetectMultiBackend(weights_path, device=device)
stride = model.stride

names = model.names


YOLOv5  2025-7-8 Python-3.10.18 torch-2.7.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7088338 parameters, 0 gradients, 16.0 GFLOPs


In [11]:
'''import torch
import cv2

# Load trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inference
    results = model(frame)
    annotated = results.render()[0]

    # Show output
    cv2.imshow('RotSense - Live Detection', annotated)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
'''

"import torch\nimport cv2\n\n# Load trained YOLOv5 model\nmodel = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)\n\n# Start webcam\ncap = cv2.VideoCapture(0)\n\nwhile cap.isOpened():\n    ret, frame = cap.read()\n    if not ret:\n        break\n\n    # Inference\n    results = model(frame)\n    annotated = results.render()[0]\n\n    # Show output\n    cv2.imshow('RotSense - Live Detection', annotated)\n\n    if cv2.waitKey(1) & 0xFF == ord('q'):\n        break\n\ncap.release()\ncv2.destroyAllWindows()\n"

In [12]:
import pathlib
import pickle
class PosixPathUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "pathlib" and name == "PosixPath":
            return pathlib.WindowsPath
        return super().find_class(module, name)

def torch_load_patch(file_path, map_location=None):
    import torch
    with open(file_path, 'rb') as f:
        unpickler = PosixPathUnpickler(f)
        obj = unpickler.load()
    return obj


In [13]:
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):

    if ratio_pad is None: 
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]  
    coords[:, [1, 3]] -= pad[1] 
    coords[:, :4] /= gain
    coords[:, 0].clamp_(0, img0_shape[1])  
    coords[:, 1].clamp_(0, img0_shape[0])  
    coords[:, 2].clamp_(0, img0_shape[1])  
    coords[:, 3].clamp_(0, img0_shape[0]) 
    return coords


In [14]:
import torch
import pathlib
import cv2
import numpy as np

from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression
from utils.torch_utils import select_device

pathlib.PosixPath = pathlib.WindowsPath


weights_path = r"D:\deep learning\rotsense\yolov5\best.pt"
device = select_device('cpu')
model = DetectMultiBackend(weights_path, device=device)
model.eval()

stride = model.stride
names = model.names

video_path = r"D:\deep learning\rotsense\rotsense video.mp4"
cap = cv2.VideoCapture(video_path)


display_width = 960
display_height = 540

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    img = letterbox(frame, stride=stride, auto=False)[0]
    img = img.transpose((2, 0, 1))[::-1]  # BGR to RGB
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.float() / 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    
    with torch.no_grad():
        pred = model(img)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

 
    for det in pred:
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()
            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]} {conf:.2f}'
                cv2.rectangle(frame, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
                cv2.putText(frame, label, (int(xyxy[0]), int(xyxy[1]) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    resized_frame = cv2.resize(frame, (display_width, display_height))
    cv2.imshow('RotSense - Video Detection', resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


YOLOv5  2025-7-8 Python-3.10.18 torch-2.7.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7088338 parameters, 0 gradients, 16.0 GFLOPs


End of video.
